In [1]:
import pandas as pd
import numpy as np
import os
import censusgeocode

In [2]:
FPATH = 'coords_and_feat.csv'
df = pd.read_csv(FPATH)
print(df.shape)
df.head()

(5964, 19)


,Unnamed: 0,file,county_x,labels,label_scores,colors,color_pixel_fraction,color_scores,chunk,chunk_index,image_index,latitiude,longitude,projection,resolution,metadata,county_y,window_size,chunk_file
0,0,/Volumes/Cooper_TB_Drive/research/readmissions...,Washtenaw,"['White', 'Black', 'Line']","[0.9736676812171936, 0.9484156370162964, 0.787...","[(255, 255, 255)]",[1.0],[1.0],1,0,0,42.442355,-84.139114,EPSG:2898,"(2.0, 2.0)","{'driver': 'GTiff', 'dtype': 'uint8', 'nodata'...",Washtenaw,1000,1
1,1,/Volumes/Cooper_TB_Drive/research/readmissions...,Washtenaw,"['White', 'Black', 'Line']","[0.9736676812171936, 0.9484156370162964, 0.787...","[(255, 255, 255)]",[1.0],[1.0],1,1,1,42.442340,-84.131704,EPSG:2898,"(2.0, 2.0)","{'driver': 'GTiff', 'dtype': 'uint8', 'nodata'...",Washtenaw,1000,1
2,2,/Volumes/Cooper_TB_Drive/research/readmissions...,Washtenaw,"['White', 'Black', 'Line']","[0.9736676812171936, 0.9484156370162964, 0.787...","[(255, 255, 255)]",[1.0],[1.0],1,10,10,42.442182,-84.065016,EPSG:2898,"(2.0, 2.0)","{'driver': 'GTiff', 'dtype': 'uint8', 'nodata'...",Washtenaw,1000,1
3,3,/Volumes/Cooper_TB_Drive/research/readmissions...,Washtenaw,"['White', 'Black', 'Line']","[0.9736676812171936, 0.9484156370162964, 0.787...","[(255, 255, 255)]",[1.0],[1.0],1,100,100,42.436567,-84.020588,EPSG:2898,"(2.0, 2.0)","{'driver': 'GTiff', 'dtype': 'uint8', 'nodata'...",Washtenaw,1000,1
4,4,/Volumes/Cooper_TB_Drive/research/readmissions...,Washtenaw,"['White', 'Black', 'Line']","[0.9736676812171936, 0.9484156370162964, 0.787...","[(255, 255, 255)]",[1.0],[1.0],1,101,101,42.436545,-84.013179,EPSG:2898,"(2.0, 2.0)","{'driver': 'GTiff', 'dtype': 'uint8', 'nodata'...",Washtenaw,1000,1


In [3]:
vintages = [
    'Current_Current',
    'Census2010_Current',
    'ACS2013_Current',
    'ACS2014_Current',
    'ACS2015_Current',
    'ACS2016_Current',
    'ACS2017_Current',
    'ACS2018_Current',
    'Current_ACS2017',
    'Census2010_ACS2017',
    'ACS2013_ACS2017',
    'ACS2014_ACS2017',
    'ACS2015_ACS2017',
    'ACS2016_ACS2017',
    'ACS2017_ACS2017',
    'ACS2018_ACS2018',
    'Census2000_Census2010',
    'Census2010_Census2010',
]

benchmarks = ['Public_AR_Current', 'Public_AR_ACS2017', 'Public_AR_Census2010']

In [4]:
%%time
"""
get census identifiers
"""
PRINT_EVERY = 200
BENCH = 'Public_AR_Current'
VINT = 'ACS2015_Current'
cg = censusgeocode.CensusGeocode(benchmark=BENCH, vintage=VINT)

new_rows = []
errors = []

for idx, row in df.iterrows():

    if idx % PRINT_EVERY == 0:
        print(f"working on {idx} to {idx + PRINT_EVERY}...")

    lat = float(row['latitiude'])
    lon = float(row['longitude'])
    
    try:
        result = cg.coordinates(x=lon, y=lat)
        cenrow = result['Census Tracts'][0]
        cenrow['image_index'] = row['image_index']
        new_rows.append(cenrow)
    except Exception as e:
        errors.append(row['image_index'])
        print(f"ERROR at {idx} img-{row['image_index']}, ({lat:.2f}, {lon:.2f}): {e}")

census_df = pd.DataFrame(new_rows)
print(census_df.shape)
census_df.head()


working on 0 to 200...
working on 200 to 400...
working on 400 to 600...
working on 600 to 800...
working on 800 to 1000...
working on 1000 to 1200...
working on 1200 to 1400...
working on 1400 to 1600...
working on 1600 to 1800...
working on 1800 to 2000...
working on 2000 to 2200...
working on 2200 to 2400...
working on 2400 to 2600...
working on 2600 to 2800...
working on 2800 to 3000...
working on 3000 to 3200...
working on 3200 to 3400...
working on 3400 to 3600...
working on 3600 to 3800...
working on 3800 to 4000...
working on 4000 to 4200...
working on 4200 to 4400...
working on 4400 to 4600...
working on 4600 to 4800...
working on 4800 to 5000...
working on 5000 to 5200...
working on 5200 to 5400...
working on 5400 to 5600...
working on 5600 to 5800...
working on 5800 to 6000...
(5964, 20)
CPU times: user 1min 52s, sys: 7.8 s, total: 2min
Wall time: 1h 5min 57s


,GEOID,CENTLAT,AREAWATER,STATE,BASENAME,OID,LSADC,FUNCSTAT,INTPTLAT,NAME,OBJECTID,TRACT,CENTLON,AREALAND,INTPTLON,MTFCC,COUNTY,CENT,INTPT,image_index
0,26093732100,+42.4668888,2523811,26,7321,20755283161773,CT,S,+42.4719749,Census Tract 7321,57380,732100,-084.0832452,87423741,-084.0745650,G5020,093,"(-84.0832452, 42.4668888)","(-84.074565, 42.4719749)",0
1,26093732100,+42.4668888,2523811,26,7321,20755283161773,CT,S,+42.4719749,Census Tract 7321,57380,732100,-084.0832452,87423741,-084.0745650,G5020,093,"(-84.0832452, 42.4668888)","(-84.074565, 42.4719749)",1
2,26093732100,+42.4668888,2523811,26,7321,20755283161773,CT,S,+42.4719749,Census Tract 7321,57380,732100,-084.0832452,87423741,-084.0745650,G5020,093,"(-84.0832452, 42.4668888)","(-84.074565, 42.4719749)",10
3,26093733601,+42.4429685,2140691,26,7336.01,207553707208141,CT,S,+42.4443155,Census Tract 7336.01,31152,733601,-083.9803529,30603912,-083.9805350,G5020,093,"(-83.9803529, 42.4429685)","(-83.980535, 42.4443155)",100
4,26093733601,+42.4429685,2140691,26,7336.01,207553707208141,CT,S,+42.4443155,Census Tract 7336.01,31152,733601,-083.9803529,30603912,-083.9805350,G5020,093,"(-83.9803529, 42.4429685)","(-83.980535, 42.4443155)",101


In [6]:
print(len(errors))

0


In [7]:
"""
Re-running analysis on missed API calls
"""
recall_rows = []

for img_ind in errors:
    row = df[df['image_index'] == img_ind]
    lat = float(row['latitiude'])
    lon = float(row['longitude'])
    print(f"Re-running img-{img_ind}, ({lat:.2f}, {lon:.2f})")

    print(f"Re-running img-{img_ind}, ({lat:.2f}, {lon:.2f})")

    result = cg.coordinates(x=lon, y=lat)
    cenrow = result['Census Tracts'][0]
    cenrow['image_index'] = row['image_index']
    recall_rows.append(cenrow)


recall_df = pd.DataFrame(recall_rows)
print(recall_df.shape)
recall_df.head()

(0, 0)


""


In [8]:
tmp = pd.concat([census_df, recall_df])
print(tmp.shape)
tmp.to_csv(f'{BENCH}_{VINT}_by_image.csv')

tmp.head()

(5964, 20)


,GEOID,CENTLAT,AREAWATER,STATE,BASENAME,OID,LSADC,FUNCSTAT,INTPTLAT,NAME,OBJECTID,TRACT,CENTLON,AREALAND,INTPTLON,MTFCC,COUNTY,CENT,INTPT,image_index
0,26093732100,+42.4668888,2523811,26,7321,20755283161773,CT,S,+42.4719749,Census Tract 7321,57380,732100,-084.0832452,87423741,-084.0745650,G5020,093,"(-84.0832452, 42.4668888)","(-84.074565, 42.4719749)",0
1,26093732100,+42.4668888,2523811,26,7321,20755283161773,CT,S,+42.4719749,Census Tract 7321,57380,732100,-084.0832452,87423741,-084.0745650,G5020,093,"(-84.0832452, 42.4668888)","(-84.074565, 42.4719749)",1
2,26093732100,+42.4668888,2523811,26,7321,20755283161773,CT,S,+42.4719749,Census Tract 7321,57380,732100,-084.0832452,87423741,-084.0745650,G5020,093,"(-84.0832452, 42.4668888)","(-84.074565, 42.4719749)",10
3,26093733601,+42.4429685,2140691,26,7336.01,207553707208141,CT,S,+42.4443155,Census Tract 7336.01,31152,733601,-083.9803529,30603912,-083.9805350,G5020,093,"(-83.9803529, 42.4429685)","(-83.980535, 42.4443155)",100
4,26093733601,+42.4429685,2140691,26,7336.01,207553707208141,CT,S,+42.4443155,Census Tract 7336.01,31152,733601,-083.9803529,30603912,-083.9805350,G5020,093,"(-83.9803529, 42.4429685)","(-83.980535, 42.4443155)",101


In [9]:
tmp['image_index'] = tmp['image_index'].astype(int)
tmp.dtypes

GEOID          object
CENTLAT        object
AREAWATER       int64
STATE          object
BASENAME       object
OID             int64
LSADC          object
FUNCSTAT       object
INTPTLAT       object
NAME           object
OBJECTID        int64
TRACT          object
CENTLON        object
AREALAND        int64
INTPTLON       object
MTFCC          object
COUNTY         object
CENT           object
INTPT          object
image_index     int64
dtype: object

In [10]:
merged = df.merge(tmp, how='left', on='image_index')
print(merged.shape)
merged['Census_Benchmark'] = BENCH
merged['Census_Vintage'] = VINT
print(merged.columns)
merged.to_csv('CLEAN_DATA.csv', index=False)
merged.head()

(5964, 38)
Index(['Unnamed: 0', 'file', 'county_x', 'labels', 'label_scores', 'colors',
       'color_pixel_fraction', 'color_scores', 'chunk', 'chunk_index',
       'image_index', 'latitiude', 'longitude', 'projection', 'resolution',
       'metadata', 'county_y', 'window_size', 'chunk_file', 'GEOID', 'CENTLAT',
       'AREAWATER', 'STATE', 'BASENAME', 'OID', 'LSADC', 'FUNCSTAT',
       'INTPTLAT', 'NAME', 'OBJECTID', 'TRACT', 'CENTLON', 'AREALAND',
       'INTPTLON', 'MTFCC', 'COUNTY', 'CENT', 'INTPT', 'Census_Benchmark',
       'Census_Vintage'],
      dtype='object')


,Unnamed: 0,file,county_x,labels,label_scores,colors,color_pixel_fraction,color_scores,chunk,chunk_index,...,TRACT,CENTLON,AREALAND,INTPTLON,MTFCC,COUNTY,CENT,INTPT,Census_Benchmark,Census_Vintage
0,0,/Volumes/Cooper_TB_Drive/research/readmissions...,Washtenaw,"['White', 'Black', 'Line']","[0.9736676812171936, 0.9484156370162964, 0.787...","[(255, 255, 255)]",[1.0],[1.0],1,0,...,732100,-084.0832452,87423741,-084.0745650,G5020,093,"(-84.0832452, 42.4668888)","(-84.074565, 42.4719749)",Public_AR_Current,ACS2015_Current
1,1,/Volumes/Cooper_TB_Drive/research/readmissions...,Washtenaw,"['White', 'Black', 'Line']","[0.9736676812171936, 0.9484156370162964, 0.787...","[(255, 255, 255)]",[1.0],[1.0],1,1,...,732100,-084.0832452,87423741,-084.0745650,G5020,093,"(-84.0832452, 42.4668888)","(-84.074565, 42.4719749)",Public_AR_Current,ACS2015_Current
2,2,/Volumes/Cooper_TB_Drive/research/readmissions...,Washtenaw,"['White', 'Black', 'Line']","[0.9736676812171936, 0.9484156370162964, 0.787...","[(255, 255, 255)]",[1.0],[1.0],1,10,...,732100,-084.0832452,87423741,-084.0745650,G5020,093,"(-84.0832452, 42.4668888)","(-84.074565, 42.4719749)",Public_AR_Current,ACS2015_Current
3,3,/Volumes/Cooper_TB_Drive/research/readmissions...,Washtenaw,"['White', 'Black', 'Line']","[0.9736676812171936, 0.9484156370162964, 0.787...","[(255, 255, 255)]",[1.0],[1.0],1,100,...,733601,-083.9803529,30603912,-083.9805350,G5020,093,"(-83.9803529, 42.4429685)","(-83.980535, 42.4443155)",Public_AR_Current,ACS2015_Current
4,4,/Volumes/Cooper_TB_Drive/research/readmissions...,Washtenaw,"['White', 'Black', 'Line']","[0.9736676812171936, 0.9484156370162964, 0.787...","[(255, 255, 255)]",[1.0],[1.0],1,101,...,733601,-083.9803529,30603912,-083.9805350,G5020,093,"(-83.9803529, 42.4429685)","(-83.980535, 42.4443155)",Public_AR_Current,ACS2015_Current
